<a href="https://colab.research.google.com/github/VamsiReddy-hub/LMS_ASSIGNMENTS/blob/main/13_02_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
import pandas as pd

# Try to read the CSV file with error handling and quoting adjustments
try:
    # Replace 'error_bad_lines' with 'on_bad_lines' and set it to 'skip' to skip bad lines
    data = pd.read_csv("/content/fake_news.csv", quoting=3, on_bad_lines='skip')
    # quoting=3 (csv.QUOTE_NONE) tells pandas to not consider any character as a quote
except pd.errors.ParserError as e:
    print(f"An error occurred: {e}")
    # If still facing issues, print the problematic line number for manual inspection
    print(f"Error occurred around line: {e.lineno()}")
    # You may need to open the CSV file in a text editor and review the lines around the reported line number to identify and correct the issue.

data.head()
data=pd.read_csv("/content/fake_news.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
data.shape

(21038, 5)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21038 entries, 0 to 21037
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      21038 non-null  int64 
 1   title   20475 non-null  object
 2   author  19063 non-null  object
 3   text    20999 non-null  object
 4   label   21038 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 821.9+ KB


In [10]:
data.isna().sum()

,0
id,0
title,563
author,1975
text,39
label,0


In [11]:
data=data.drop(['id'], axis=1)

In [12]:
data=data.fillna('')

In [13]:
data['content'] = data['author']+' '+data['title']+' '+data['text']

In [14]:
data=data.drop(['title','author','text'],axis=1)

In [15]:
data.head()

,label,content
0,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,1,Consortiumnews.com Why the Truth Might Get You...
3,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,1,Howard Portnoy Iranian woman jailed for fictio...


In [16]:
data['content']=data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [17]:
data['content']=data['content'].str.replace('[\w\s]','')

In [18]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
data['content']=data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [20]:
!pip install textblob

In [23]:
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from textblob import Word

data['content'] = data['content'].apply(lambda x : " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'].head()

[nltk_data] Downloading package wordnet to /root/nltk_data...


,content
0,darrell lucus house dem aide: we didn’t even s...
1,"daniel j. flynn flynn: hillary clinton, big wo..."
2,consortiumnews.com why the truth might get you...
3,jessica purkiss 15 civilian killed in single u...
4,howard portnoy iranian woman jailed for fictio...


In [24]:
X = data[['content']]
y = data['label']

In [25]:
from sklearn.model_selection import train_test_split

In [27]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=45,stratify=y)
# Changed 'x' to 'X' to use the DataFrame created earlier

In [28]:
print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)

(14726, 1)
(14726,)
(6312, 1)
(6312,)


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(data['content'])
xtrain_tfidf = tfidf_vect.transform(x_train['content'])
xtest_tfidf = tfidf_vect.transform(x_test['content'])

In [33]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
pclf = PassiveAggressiveClassifier()

pclf.fit(xtrain_tfidf, y_train)
predictions = pclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3154
           1       0.97      0.96      0.96      3158

    accuracy                           0.96      6312
   macro avg       0.96      0.96      0.96      6312
weighted avg       0.96      0.96      0.96      6312



In [34]:
print(metrics.confusion_matrix(y_test,predictions))

[[3055   99]
 [ 124 3034]]


In [37]:
from sklearn.neural_network import MLPClassifier
mcpclf = MLPClassifier(hidden_layer_sizes=(256,64,16),
                       activation = 'relu',
                        solver = 'adam')
mcpclf.fit(xtrain_tfidf, y_train)
predictions = mcpclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test, predictions))
mcpclf.fit(xtrain_tfidf, y_train)
predictions = mcpclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3154
           1       0.97      0.96      0.96      3158

    accuracy                           0.96      6312
   macro avg       0.96      0.96      0.96      6312
weighted avg       0.96      0.96      0.96      6312

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3154
           1       0.97      0.96      0.96      3158

    accuracy                           0.96      6312
   macro avg       0.96      0.96      0.96      6312
weighted avg       0.96      0.96      0.96      6312

